# ML в Биологии
## MDE

# Минимальный детектируемый эффект

## Теория

### Введение

**MDE** := Minimum Detectable Effect (минимальный эффект, полученный от изменений в тестовой группе, который может быть обнаружен с заданной уверенностью (мощностью) в эксперименте).

Обозначения:
*   $\alpha$ &mdash; уровень значимости,
*   $\beta$ &mdash; мощность,
*   $n$ &mdash; размер группы,
*   $\varepsilon$ &mdash; величина эффекта.

При дизайне эксперимента:
1.   Задано: $\alpha, \beta, n \implies$ Какой минимальный эффект $\varepsilon$ можем задетектировать?
2.   Задано: $\alpha, \beta, \varepsilon \implies$ Какой минимальный размер каждой из групп нам необходим?





### Что в основе


$X_1...X_n$ &mdash; выборка  с параметрами $\mathsf{E} X_i = a_1$, $\mathsf{D} X_i = \sigma_1^2$

$Y_1...Y_m$ &mdash; выборка с параметрами $\mathsf{E} Y_i = a_2$, $\mathsf{D} Y_i = \sigma_2^2$

---
Гипотезы:
$\mathsf{H}_0\colon a_1 = a_2\ \ vs.\ \ \mathsf{H}_1\colon a_1 > a_2$\
Статистика:
$T(\overline{X}, \overline{Y}) = f\left(\overline{X} - \overline{Y}\right)$\
Критерий: $S = \left\{T(\overline{X}, \overline{Y}) > z_{1-\alpha}\right\}$.



При справедливости $\mathsf{H}_0$ получаем\
$$T(\overline{X}, \overline{Y}) = \frac{\overline{X} - \overline{Y}}{\sqrt{\frac{\sigma_1^2}{n} + \frac{\sigma_2^2}{m}}} \stackrel{\mathsf{H}_0}{\approx} \mathcal{N}(0, 1)$$.


### Случай наличия эффекта


$X_1...X_n$ &mdash; выборка   $\mathsf{E}X_i = a_1$,
$\mathsf{D}X_i = \sigma_1^2$\
$Y_1...Y_m$ &mdash; выборка   $\mathsf{E}Y_i = a_2$,
$\mathsf{D}Y_i = \sigma_2^2$

Гипотезы:
$\mathsf{H}_0\colon a_1 = a_2\  vs.\ \ \mathsf{H}_1\colon a_1 > a_2$

---

Пусть $\varepsilon = (a_1 - a_2)$ &mdash; эффект.\
$$\overline{X} - \overline{Y} \stackrel{\mathsf{H}_1}{\approx} \mathcal{N}\left(\varepsilon, \frac{\sigma_1^2}{n} + \frac{\sigma_2^2}{m}\right)$$.\
Тогда из условий на уровень значимости и мощность:
$$\varepsilon \ge \left( z_{1-\alpha} + z_{\beta}\right) \sqrt{\frac{\sigma_1^2}{n} + \frac{\sigma_2^2}{m}} \stackrel{n=m}{=}\left( z_{1-\alpha} + z_{\beta}\right) \sqrt{\frac{\sigma_1^2 +\sigma_2^2}{n}}$$

где $z_{1-\alpha}$ и $z_{\beta}$ &mdash; соответствующие квантили нормального распределения, $n$ &mdash; размер выборки, $\sigma_1^2$ и $\sigma_2^2$ - дисперсии.

### Итоговый ответ


$$\varepsilon_{min} = MDE = \left( z_{1-\alpha} + z_{\beta}\right) \sqrt{\frac{\sigma_1^2 +\sigma_2^2}{n}}$$  
  
Также можно совершить обратную операцию: зная требуемое значение эффекта, получить необходимое количество объектов в каждой из групп по следующей формуле:

$$n = \left( z_{1-\alpha} + z_{\beta}\right)^2 \frac{\sigma_1^2 +\sigma_2^2}{\varepsilon^2}$$

## Практика

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

import matplotlib.pyplot as plt
import scipy.stats as sps

sns.set_style("darkgrid")
sns.set(palette='Set2', font_scale=1.5)

### 1. Визуализация теоретических зависимостей

Рассмотрим зависимости MDE и размера выборки от дисперсии, желаемых ошибки первого рода и мощности критерия. Для начала зафиксируем значения переменных (будем считать, что мы из исторических данных или теоретических предположений знаем характерную величину дисперсии каждой из выборок).

In [ ]:
var_x = var_y = 1  # Дисперсии выборок
sample_size = 100  # Размер выборки
effect = 0.1  # Размер эффекта
alpha = 0.05  # Уровень значимости
betas = np.array([0.4, 0.6, 0.8, 0.99]).reshape(-1, 1)  # Значения мощности

Реализуйте функции подбора размера выборки и расчета MDE в соответствии с выведенными формулами. Обратите внимание, что величина дисперсии не требует расчета, а передается напрямую в функцию. Вам может потребоваться метод `sps.norm.ppf`.

Подумайте, в какую сторону стоит округлять результат для минимально необходимого объема выборки.

In [ ]:
def get_mde(alpha, beta, sample_size, var_x, var_y=var_x):
    '''Расчитывает MDE

    Аргументы:
    alpha -- желаемая ошибка первого рода
    beta -- желаемая мощность
    sample_size -- размер выборок
    var_x, var_y -- дисперсии выборок

    Возвращает:
    mde -- теоретический MDE
    '''

    q_sum = sps.norm.ppf(1 - alpha) + sps.norm.ppf(beta)
    mde = q_sum / np.sqrt(sample_size) * np.sqrt(var_x + var_y)

    return mde

In [ ]:
def get_sample_size(alpha, beta, mde, var_x, var_y=var_x):
    '''Расчитывает количество объектов в группе для детектирование MDE

    Аргументы:
    alpha -- желаемая ошибка первого рода
    beta -- желаемая мощность
    mde -- необходимый эффект
    var_x, var_y -- дисперсии выборок

    Возвращает:
    n -- теоретический минимальный размер выборки
    '''

    q_sum = sps.norm.ppf(1 - alpha) + sps.norm.ppf(beta)
    n = np.ceil((q_sum ** 2) * (var_x + var_y) / (mde * mde)).astype(int)

    return n

Также сделаем вспомогательную функцию для визуализации результатов

In [ ]:
def plot(grid, mdes, sample_sizes, title='', third_dimension=None, label=''):
    ''' Строит графики MDE и размера выборки по сетке значений. '''

    _, axes = plt.subplots(1, 2, figsize=(20, 8))

    plt.suptitle(title, fontsize=20)

    if third_dimension is not None:
        for mde, sample_size, third in zip(mdes, sample_sizes, third_dimension):
            axes[0].plot(grid.reshape(-1), mde, lw=5, label=f'{label} = {third[0]}')
            axes[0].legend()
            axes[1].plot(grid.reshape(-1), sample_size, lw=5)
    else:
        axes[0].plot(grid.reshape(-1), mdes, lw=5)
        axes[1].plot(grid.reshape(-1), sample_sizes, lw=5)


    axes[0].set_title('MDE')
    axes[0].set_xlabel(title, fontsize=20)
    axes[1].set_title('Размер выборки')
    axes[1].set_xlabel(title, fontsize=20)

    plt.show()

Будем исследовать зависимости следующим образом: зафиксируем все параметы эксперимента, кроме одного. Для этого выбранного параметра рассчитаем формулы в срезе по разным мощностям

*Зависимость от дисперсии*

In [ ]:
var_x_grid = np.arange(0.1, 10, 0.01).reshape(1, -1)

plot(
    var_x_grid,
    get_mde(alpha, betas, sample_size, var_x_grid),
    get_sample_size(alpha, betas, effect, var_x_grid),
    title='Дисперсия',
    third_dimension=betas,
    label='Мощность'
)

*Зависимость от ошибки I рода*

In [ ]:
alpha_grid = np.arange(1e-4, 0.5, 1e-5)

plot(
    alpha_grid,
    get_mde(alpha_grid, betas, sample_size, var_x),
    get_sample_size(alpha_grid, betas, effect, var_x),
    title='Ошибка первого рода',
    third_dimension=betas,
    label='Мощность'
)

*Зависимость от мощности*

In [ ]:
beta_grid = np.arange(1e-2, 1, 1e-5)

plot(
    beta_grid,
    get_mde(alpha, beta_grid, sample_size, var_x),
    get_sample_size(alpha, beta_grid, effect, var_x),
    title='Мощность',
)

*Зависимость MDE и размера выборки*

In [ ]:
sample_size_grid = np.arange(10, 500, 10)
effect_grid = np.arange(1e-2, 0.1, 1e-3)

sample_sizes = get_sample_size(alpha, betas, effect_grid, var_x)
mdes = get_mde(alpha, betas, sample_size_grid, var_x)

third_dimension = betas

_, axes = plt.subplots(1, 2, figsize=(20, 8))

for mde, ss, third in zip(mdes, sample_sizes, third_dimension):
    axes[0].plot(sample_size_grid.reshape(-1), mde, lw=3, label=f'Мощность = {third[0]}')
    axes[0].legend()
    axes[1].plot(effect_grid.reshape(-1), ss, lw=3)


axes[0].set_title('MDE')
axes[0].set_xlabel('Размер выборки')
axes[1].set_title('Размер выборки')
axes[1].set_xlabel('MDE')

plt.show()

Теперь рассмотрим парную зависимость, а именно, как мощность и ошибка первого рода совместно влияют на MDE и размер выборки.

In [ ]:
alpha_grid = np.arange(1e-2, 1, 1e-2).reshape(-1, 1)
beta_grid = np.arange(1e-2, 1, 1e-2)

mdes = get_mde(alpha_grid, beta_grid, sample_size, var_x)
sample_sizes = get_sample_size(alpha_grid, beta_grid, effect, var_x)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(25, 10))

ticks = [0, 20, 40, 60, 80, 99]
ticklabels = alpha_grid.round(2).reshape(-1).tolist()[::20] + [alpha_grid[-1][0]]

axes[0].set_title('MDE')
sns.heatmap(mdes, ax=axes[0])

axes[1].set_title('Размер выборки')
sns.heatmap(sample_sizes, ax=axes[1])

for i in [0, 1]:
    axes[i].set_yticks(ticks)
    axes[i].set_yticklabels(ticklabels)
    axes[i].set_ylabel('Ошибка первого рода', fontsize=20)
    axes[i].set_xticks(ticks)
    axes[i].set_xticklabels(ticklabels)
    axes[i].set_xlabel('Мощность', fontsize=20)

plt.show()

Опишите все наблюдаемые зависимости.

**Вывод:**

1. **MDE и размер выборки**:

MDE увеличивается с ростом дисперсии, мощности теста $1-\beta$ и снижением уровня ошибки первого рода $\alpha$. Размер выборки растет при увеличении дисперсии, мощности теста и снижении $\alpha$.

2. **Взаимозависимость параметров**:

Увеличение размера выборки снижает MDE, позволяя обнаруживать меньшие эффекты. Для уменьшения MDE требуется экспоненциальный рост выборки.

3. **Оптимизация экспериментов**:

Снижение дисперсии данных снижает требования к выборке и минимальному эффекту. Выбор более высокого значения $\alpha$ или меньшей мощности позволяет сократить размер выборки. При проектировании тестов важно учитывать ограничения ресурсов и значимость результата.

### 2. Планирование эксперимента

Перед непосредственным проведением эксперимента параметры используемого критерия необходимо зафиксировать. Если для ошибки первого рода обычно берется значение $0.05$, дисперсия зависит от природы выборок, то величины других параметров нуждаются в подборе. Для этого подойдет следующая таблица:
- по строкам отложены значения детектируемого эффекта
- по столбцам отложены значения мощности
- в ячейках находятся рассчитанные размеры выборок, соответствующие заданным MDE и мощности

Теперь для выбора значений нужно понять размен между этими величинами. Например, насколько большую выборку исследователь может себе позволить: например, сколько человек может участовать в AB-тестировании и как долго. MDE также необходимо выбирать исходя из практической значимости эксперимента.

Для примера рассмотрим построение подобной таблицы для $\alpha=0.05$ и $\sigma_1 = \sigma_2 = 1$:

In [ ]:
mde_grid = np.arange(0.01, 0.1, 0.01).reshape(-1, 1)
beta_grid = np.arange(0.1, 1, 0.1)

In [ ]:
sample_sizes = get_sample_size(alpha, beta_grid, mde_grid, var_x)
df = pd.DataFrame(sample_sizes, columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))
df.columns.name = 'beta'
df.index.name = 'mde'

df

Теперь попробуем провести эксперимент. Допустим мы хотим иметь $MDE=0.05$ и мощность $\beta=0.8$. Оцените, какой размер выборки нам необходимо взять?

In [ ]:
mde = 0.05
beta = 0.8

sample_size = get_sample_size(alpha=0.05, beta=beta, mde=mde, var_x=1)
print(f'Размер контрольной и тестовой групп: {sample_size}')

Посмотрим, как ведет себя мощность критерия в зависимости от реального эффекта, выраженного статистикой $T(\overline{X}, \overline{Y}) = \left(\overline{X} - \overline{Y}\right)$.

Напомним, что критерий имеет вид: $S = \left\{T(\overline{X}, \overline{Y}) \geq с_{\alpha}\right\}$. Для начала рассчитаем константу критерия.

In [ ]:
from scipy.stats import norm

alpha = 0.05
c_alpha = norm.ppf(1 - alpha)

Далее будем оценивать мощность как долю отверженных гипотез на случайных запусках.

Для этого сгенерируйте две выборки (контрольную и тестовую со смещением) из нормального распределения размера `(iters_count, sample_size)`, усредните каждую из них по `axis=1`, посчитайте разность между получившимися средними и сравните с константой критерия. Посчитайте долю отвержения. Повторите для нескольких значений смещения и постройте график мощности от смещения (эффекта).

In [ ]:
real_effect_grid = np.arange(-0.1, 0.3, 0.01) # смещение
iters_count = 10_000

In [ ]:
power = []
for real_effect in tqdm(real_effect_grid):
    # сгенерирйте выборки
    control_sample = np.random.normal(loc=0, scale=1, size=(iters_count, sample_size))
    test_sample = np.random.normal(loc=real_effect, scale=1, size=(iters_count, sample_size))

    # найдите среднее
    control_sample_mean = control_sample.mean(axis=1)
    test_sample_mean = test_sample.mean(axis=1)

    T_values = test_sample_mean - control_sample_mean  # Разности средних

    # посчитайте долю отвержения гипотезы
    rejects = np.mean(T_values >= c_alpha)

    power.append(rejects)

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(real_effect_grid, power, lw=3)

plt.hlines(
    beta, real_effect_grid.min(), real_effect_grid.max(),
    color='red', label='Теоретические'
)

plt.xlabel('Реальный эффект', fontsize=20)
plt.ylabel('Реальная мощность', fontsize=20)

plt.legend()
plt.show()

**Вывод:**

Когда эффект (смещение) близок к нулю, тест редко отклоняет нулевую гипотезу, что приводит к низкой мощности. С увеличением реального эффекта мощность теста возрастает. Это делает тест более чувствительным к эффекту, увеличивая вероятность отклонения нулевой гипотезы.

Для достижения высокой мощности теста (например, 80%), реальный эффект должен быть значительным. Это иллюстрирует, как размер эффекта влияет на способность теста обнаруживать различия между группами.